# Test the split
Notebook for testing SelectSplitData and comparing it with the ValidationSplit

In [4]:
%load_ext autoreload
%autoreload 2

import sys

# setting path
sys.path.append('../')

from modules.TransformApplier import TransformApplier
from modules.Wav2Spec import Wav2Spec
from modules.SimpleDataset import SimpleDataset
from modules.PretrainedModel import *
from modules.OnlyXTransform import OnlyXTransform
from modules.SelectSplitData import *
from modules.SimpleAttention import *

import pandas as pd
import torch.nn as nn
import torch
import timm
import json
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np



In [72]:
def collate_fn(data):
    """
    Define how the DataLoaders should batch the data
    """
    max_dim = max([d[0].shape[-1] for d in data])
    pad_x = lambda x: torch.concat([x, torch.zeros((max_dim - x.shape[-1], ))])
    return torch.stack([pad_x(d[0]) for d in data], axis=0), torch.stack([torch.tensor(d[1]) for d in data])
    

In [63]:
DATA_PATH = '../data/'
metadata = pd.read_csv(f'{DATA_PATH}train_metadata.csv')# [:4000]

with open(f'{DATA_PATH}all_birds.json') as f:
    birds = json.load(f)

In [64]:
tts = metadata.sample(frac=.05).index # train test split
df_val = metadata.iloc[tts]
df_train = metadata.drop(axis=0,index=tts)

In [67]:
train_data = SimpleDataset(df_train, DATA_PATH, mode='train', labels=birds)
val_data = SimpleDataset(df_val, DATA_PATH, mode='train', labels=birds)

In [71]:
len(val_data)

743

In [73]:
bs = 16
train_loader = DataLoader(train_data, batch_size=bs, num_workers=4, collate_fn=collate_fn, shuffle=True)
val_loader = DataLoader(val_data, batch_size=bs, num_workers=4, collate_fn=collate_fn)
# 1 row per batch 47 for val, 882 for train

In [80]:
duration = 30
n_splits = 5
transforms1 = TransformApplier([nn.Identity(), SelectSplitData(duration, n_splits)])
data_pipeline_val = nn.Sequential(
    transforms1, 
).to(device)
